In [16]:
import numpy as np
import pandas as pd
import random
from scipy import sparse
from scipy.sparse import vstack

from sklearn import model_selection, preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.linear_model import LogisticRegression, BayesianRidge, LinearRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, log_loss, classification_report

import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
from geopy.distance import vincenty

# Any results you write to the current directory are saved as output.

In [3]:
data_path = "../input/"
train_file =  "train.json"
test_file =  "test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)

(49352, 15)
(74659, 14)


In [4]:
list(train_df.columns.values)

['bathrooms',
 'bedrooms',
 'building_id',
 'created',
 'description',
 'display_address',
 'features',
 'interest_level',
 'latitude',
 'listing_id',
 'longitude',
 'manager_id',
 'photos',
 'price',
 'street_address']

In [5]:
cols_to_delete = ['photos','building_id','street_address','display_address']
train_df.drop(cols_to_delete,inplace=True,axis=1)

In [6]:
train_df.drop(cols_to_delete,inplace=True,axis=1)

In [11]:
cen_longitude=train_df['longitude'].mean()

In [12]:
cen_latitude=train_df['latitude'].mean()

In [13]:
len(train_df)

49352

In [196]:
def distance(x,y):
    centroid = (cen_latitude, cen_longitude)
    point=(x,y)
    return (vincenty(centroid, point).miles)*100

In [18]:
a=[]
for index, row in train_df.iterrows():
    a.append(distance(row['latitude'],row['longitude']))

In [21]:
train_df

,bathrooms,bedrooms,created,description,features,interest_level,latitude,listing_id,longitude,manager_id,price,Distance
10,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,3000,1.990931
10000,1.0,2,2016-06-12 12:19:27,,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,5465,3.712856
100004,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...","[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,2850,2.426099
100007,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,3275,1.059418
100013,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,3350,5.706492
100014,2.0,4,2016-04-19 04:24:47,,[],medium,40.7429,6894514,-74.0028,b209e2c4384a64cc307c26759ee0c651,7995,2.472872
100016,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,"[prewar, elevator, Dogs Allowed, Cats Allowed,...",low,40.8012,6930771,-73.9660,01287194f20de51872e81f660def4784,3600,4.151589
100020,2.0,1,2016-04-13 06:01:42,"This huge sunny ,plenty of lights 1 bed/2 bath...","[Doorman, Elevator, Pre-War, Terrace, Laundry ...",low,40.7427,6867392,-73.9957,e6472c7237327dd3903b3d6f6a94515a,5645,2.099995
100026,1.0,1,2016-04-20 02:36:35,<p><a website_redacted,"[Cats Allowed, Dogs Allowed, Elevator, Laundry...",medium,40.8234,6898799,-73.9457,c1a6598437b7db560cde66e5a297a53f,1725,5.672636
100027,2.0,4,2016-04-02 02:58:15,This is a spacious four bedroom with every bed...,"[Dishwasher, Hardwood Floors]",low,40.7278,6814332,-73.9808,23a01ea7717b38875f5b070282d1b9d2,5800,1.622670


In [91]:
def grades_average(my_list):
    sum_of_grades = sum(my_list)
    average = sum_of_grades / len(my_list)
    return average

In [20]:
train_df["Distance"]=a

In [92]:
print(grades_average(a))

4.346265908990207


In [99]:
k=pd.DataFrame(a)
k.hist()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1344b64e0>]], dtype=object)

In [102]:
pd.DataFrame(a).describe()

,0
count,49352.000000
mean,4.346266
std,84.928739
min,0.075857
25%,1.837859
50%,2.466716
75%,3.526085
max,5383.451577


In [137]:
np.sum(m<10000000)

0    49352
dtype: int64

In [75]:
dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
dataset.loc[ dataset['Age'] > 64, 'Age']   

TypeError: unhashable type: 'slice'

In [42]:
train_df[]

,bathrooms,bedrooms,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,price
10,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,3000
10000,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,5465
100004,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,2850
100007,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,3275
100013,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,3350
100014,2.0,4,2016-04-19 04:24:47,,West 18th Street,[],medium,40.7429,6894514,-74.0028,b209e2c4384a64cc307c26759ee0c651,7995
100016,1.0,2,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,"[prewar, elevator, Dogs Allowed, Cats Allowed,...",low,40.8012,6930771,-73.9660,01287194f20de51872e81f660def4784,3600
100020,2.0,1,2016-04-13 06:01:42,"This huge sunny ,plenty of lights 1 bed/2 bath...",West 21st Street,"[Doorman, Elevator, Pre-War, Terrace, Laundry ...",low,40.7427,6867392,-73.9957,e6472c7237327dd3903b3d6f6a94515a,5645
100026,1.0,1,2016-04-20 02:36:35,<p><a website_redacted,Hamilton Terrace,"[Cats Allowed, Dogs Allowed, Elevator, Laundry...",medium,40.8234,6898799,-73.9457,c1a6598437b7db560cde66e5a297a53f,1725
100027,2.0,4,2016-04-02 02:58:15,This is a spacious four bedroom with every bed...,522 E 11th,"[Dishwasher, Hardwood Floors]",low,40.7278,6814332,-73.9808,23a01ea7717b38875f5b070282d1b9d2,5800


In [41]:
train_df['display_address'].unique()

array(['Metropolitan Avenue', 'Columbus Avenue', 'W 13 Street', ...,
       'West 73rd Street and Broadway', 'Ridge Blvd. ', 'Rector Pl'], dtype=object)

In [43]:
from geopy.distance import vincenty
newport_ri = (40.7145, -73.9425)
cleveland_oh = (40.7539, -73.9677)
print(vincenty(newport_ri, cleveland_oh).miles)

3.0233912074325597


In [37]:
train_df["description"]

10        A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...
10000                                                      
100004    Top Top West Village location, beautiful Pre-w...
100007    Building Amenities - Garage - Garden - fitness...
100013    Beautifully renovated 3 bedroom flex 4 bedroom...
100014                                                     
100016    Stunning unit with a great location and lots o...
100020    This huge sunny ,plenty of lights 1 bed/2 bath...
100026                             <p><a  website_redacted 
100027    This is a spacious four bedroom with every bed...
100030    New to the market! Spacious studio located in ...
10004     Check out this one bedroom apartment in a grea...
100044    ***LOW FEE. Beautiful CHERRY OAK WOODEN FLOORS...
100048    Lincoln Square's premier full service building...
10005     Spacious 1-Bedroom to fit King-sized bed comfo...
100051    Stunning  full renovated studio unit. High cei...
100052    East Village is a great commun

In [26]:
k="2016-06-24 07:54:24"

In [28]:
k.split('-')[0]

'2016'

In [24]:
train_df['created']

10        2016-06-24 07:54:24
10000     2016-06-12 12:19:27
100004    2016-04-17 03:26:41
100007    2016-04-18 02:22:02
100013    2016-04-28 01:32:41
100014    2016-04-19 04:24:47
100016    2016-04-27 03:19:56
100020    2016-04-13 06:01:42
100026    2016-04-20 02:36:35
100027    2016-04-02 02:58:15
100030    2016-04-14 01:10:30
10004     2016-06-03 03:21:22
100044    2016-04-19 05:37:25
100048    2016-04-09 01:22:11
10005     2016-06-01 03:11:01
100051    2016-04-18 02:36:00
100052    2016-04-22 05:57:16
100053    2016-04-19 03:36:28
100055    2016-04-20 04:00:32
100058    2016-04-09 03:49:42
100062    2016-04-12 02:39:45
100063    2016-04-07 02:11:45
100065    2016-04-13 05:17:43
100066    2016-04-17 02:26:52
10007     2016-06-07 04:39:56
100071    2016-04-27 06:43:40
100075    2016-04-25 02:29:50
100076    2016-04-22 05:39:10
100079    2016-04-21 05:27:10
100081    2016-04-26 02:59:37
                 ...         
99915     2016-04-25 01:13:37
99917     2016-04-26 04:43:14
99919     

In [32]:
year=lambda x : x.split('-')[1]

In [33]:
train_df['month']=train_df['created'].apply(year)


In [35]:
train_df['month']=n

In [22]:
capitalizer = lambda x: len(x)

In [23]:
m=train_df['features'].apply(capitalizer)

In [24]:
train_df['no_of_features']=m

In [26]:
cols_to_delete = ['latitude','longitude','manager_id','features','listing_id','description','created']
train_df.drop(cols_to_delete,inplace=True,axis=1)

In [38]:
train_df

,bathrooms,bedrooms,interest_level,price,Distance,no_of_features,month
10,1.5,3,medium,3000,1.990931,0,06
10000,1.0,2,low,5465,3.712856,5,06
100004,1.0,1,high,2850,2.426099,4,04
100007,1.0,1,low,3275,1.059418,2,04
100013,1.0,4,low,3350,5.706492,1,04
100014,2.0,4,medium,7995,2.472872,0,04
100016,1.0,2,low,3600,4.151589,8,04
100020,2.0,1,low,5645,2.099995,8,04
100026,1.0,1,medium,1725,5.672636,4,04
100027,2.0,4,low,5800,1.622670,2,04


In [42]:
train_df['price'].describe()

count    4.935200e+04
mean     3.830174e+03
std      2.206687e+04
min      4.300000e+01
25%      2.500000e+03
50%      3.150000e+03
75%      4.100000e+03
max      4.490000e+06
Name: price, dtype: float64

In [3]:
test_df[["listing_id", "display_address","bathrooms", "bedrooms", "latitude", "longitude", "price"]].head()

,listing_id,display_address,bathrooms,bedrooms,latitude,longitude,price
0,7142618,Suffolk Street,1.0,1,40.7185,-73.9865,2950
1,7210040,Thompson Street,1.0,2,40.7278,-74.0000,2850
100,7103890,101 East 10th Street,1.0,1,40.7306,-73.9890,3758
1000,7143442,South Third Street\r,1.0,2,40.7109,-73.9571,3300
100000,6860601,"Midtown West, 8th Ave",2.0,2,40.7650,-73.9845,4900


In [ ]:
train_df.loc[ train_df['Age'] <= 16, 'Age'] 					       = 0
train_df.loc[(train_df['Age'] > 16) & (train_df['Age'] <= 32), 'Age'] = 1
train_df.loc[(train_df['Age'] > 32) & (train_df['Age'] <= 48), 'Age'] = 2
train_df.loc[(train_df['Age'] > 48) & (train_df['Age'] <= 64), 'Age'] = 3
train_df.loc[ train_df['Age'] > 64, 'Age']  

In [43]:
##### import pandas as pd
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
 
 
def change_obj_to_category(df):
    i=0;
    while i<df.columns.size:
        if (df.dtypes[df.columns[i]] == 'object' or df.dtypes[df.columns[i]] == 'bool'):
            col = df.dtypes.index[i]
            df[col] = df[col].astype('category')
        i = i+1 
        
def change_int_to_float(df):
    i=0;
    while i<df.columns.size:
        if 'int' in str(df.dtypes[df.columns[i]]):
            col = df.dtypes.index[i]
            df[col] = df[col].astype('float64')
        i = i+1
       
def generateXY(df,target_col,var_list):
   
    #make a copy of the required subset and drop rows containing Na
    mdata = df[var_list+[target_col]].copy()
    mdata.dropna(inplace=True)
   
    #Separate target and put it in Y
    Y = mdata[target_col].tolist()
    del mdata[target_col]
   
    #Now, process data and create dummy variables if required with final data in Xvars
    import pandas
    Xvars = pandas.DataFrame()
    import pandas as pd
    for cols in var_list:
        if (str(mdata[cols].dtype) == 'category'):
            dummySer= pd.get_dummies(mdata[cols],prefix=cols+'_')
            Xvars = pd.concat([Xvars,dummySer],axis=1)
        else:
            Xvars =  pd.concat([Xvars,mdata[cols]],axis=1)
   
    X = Xvars.values.tolist()
   
    return X,Y
        
        
def roc_score_model(model,X,Y):
    from sklearn.metrics import roc_auc_score
    return roc_auc_score(Y,pd.DataFrame((model.predict_proba(X)))[1].tolist())
 
def feature_imp(colNames,imps):
    df = pd.DataFrame(columns=('Feature','Importance'),index=[x for x in range(0,len(colNames))])
    i = 0
    for col in colNames:
        df['Feature'][i] = col
        df['Importance'][i] = imps[i]
        i=i+1
   
    df = df.sort_values(by='Importance',ascending=False)
    return df
 
def predict_th(model,tx,ty,threshold=0.5):
    import pandas as pd
    probs = model.predict_proba(tx)[:,1].tolist()
    predictions = []
    for i in range(0,len(ty)):
        if probs[i]>threshold:
            predictions.append(1)
        else:
            predictions.append(0)
   
    return predictions 

In [49]:
print('Starting data read from csv file.')
match_data = train_df

numeric_cols=['bathrooms','bedrooms','price','Distance','no_of_features','month']
print('Deleting columns containing huge missing values, Single valued items and id columns which are of no use.')
print('No data leakage is present as such.')
match_data.interest_level = match_data.interest_level.astype('category')


print('Setting aside 20% of the data as hold_out.')
#Now, let's set 20% of the data as holdout data [test data]
from sklearn.model_selection import train_test_split
train_data, hold_out = train_test_split(match_data, train_size = 0.8,random_state=2135)

 
print('Using standard scaler of sklearn library to Standardize and scale the numeric inputs.')
#Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_data[numeric_cols] = scaler.fit_transform(train_data[numeric_cols])
hold_out[numeric_cols] = scaler.transform(hold_out[numeric_cols])
 

print('\nFinding Optimal parameters for 4 different models using GridSearch.')
target_col = 'interest_level'
varToUse = train_data.columns.tolist()
varToUse.remove('interest_level')
Train_X,Train_Y = generateXY(train_data,target_col,varToUse)
Test_X,Test_Y = generateXY(hold_out,target_col,varToUse)

Starting data read from csv file.
Deleting columns containing huge missing values, Single valued items and id columns which are of no use.
No data leakage is present as such.
Setting aside 20% of the data as hold_out.
Using standard scaler of sklearn library to Standardize and scale the numeric inputs.

Finding Optimal parameters for 4 different models using GridSearch.


In [79]:
com=match_data
com[numeric_cols] = scaler.fit_transform(com[numeric_cols])
target_col = 'interest_level'
varToUse = com.columns.tolist()
varToUse.remove('interest_level')
Train_X,Train_Y = generateXY(com,target_col,varToUse)

In [80]:
t=com['interest_level'].apply(lambda x:0 if x=="low" else 1 if x=="medium" else 2)

In [81]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(t)
num_classes = y_train.shape[1]

In [83]:
# Import necessary modules
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
# Import the SGD optimizer
from keras.optimizers import SGD
# Import EarlyStopping
from keras.callbacks import EarlyStopping


# Create list of learning rates: lr_to_test
lr_to_test = [0.0001, 0.001, 0.01,0.1]

# Loop over learning rates
lr=0.001
print('\n\nTesting model with learning rate: %f\n'%lr )

# Define early_stopping_monitor
early_stopping_monitor = EarlyStopping(patience=2)
    
    # Build new model to test, unaffected by previous models
model = Sequential()
model.add(Dense(100, activation='relu', input_shape = (6,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(3, activation='softmax'))

epochs = 30
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# Create SGD optimizer with specified learning rate: my_optimizer
    
# Fit the model
model.fit(Train_X,y_train,epochs=30,validation_split=0.05,callbacks=[early_stopping_monitor])



Testing model with learning rate: 0.001000

Train on 46884 samples, validate on 2468 samples
Epoch 1/30
46884/46884 [==============================] - 5s - loss: 0.7744 - acc: 0.6947 - val_loss: 0.7675 - val_acc: 0.6941
Epoch 2/30
46884/46884 [==============================] - 4s - loss: 0.7441 - acc: 0.6947 - val_loss: 0.7519 - val_acc: 0.6945
Epoch 3/30
46884/46884 [==============================] - 4s - loss: 0.7247 - acc: 0.6946 - val_loss: 0.7143 - val_acc: 0.6941
Epoch 4/30
46884/46884 [==============================] - 4s - loss: 0.7153 - acc: 0.6948 - val_loss: 0.7041 - val_acc: 0.6933
Epoch 5/30
46884/46884 [==============================] - 4s - loss: 0.7074 - acc: 0.6954 - val_loss: 0.7132 - val_acc: 0.6937
Epoch 6/30
46884/46884 [==============================] - 4s - loss: 0.7024 - acc: 0.6950 - val_loss: 0.7115 - val_acc: 0.6929
Epoch 7/30
46884/46884 [==============================] - 4s - loss: 0.7001 - acc: 0.6952 - val_loss: 0.7043 - val_acc: 0.6953


In [76]:
predictions = model.predict(Test_X)

# Calculate predicted probability of survival: predicted_prob_true
predicted_prob_true = predictions[:,1]

# print predicted_prob_true
print(predicted_prob_true)

[ 0.36482     0.23570202  0.16619475 ...,  0.06680176  0.03400442
  0.00403398]


In [95]:
Test_Y[1]

'low'

In [78]:
predictions[1]

array([ 0.72400552,  0.23570202,  0.04029242], dtype=float32)

In [84]:
test=test_df
cols_to_delete = ['photos','building_id','street_address','display_address']
test.drop(cols_to_delete,inplace=True,axis=1)
a=[]
for index, row in test.iterrows():
    a.append(distance(row['latitude'],row['longitude']))
test["Distance"]=a
year=lambda x : x.split('-')[1]
test['month']=test['created'].apply(year)
capitalizer = lambda x: len(x)
m=test['features'].apply(capitalizer)
test['no_of_features']=m
cols_to_delete = ['latitude','longitude','manager_id','features','listing_id','description','created']
test.drop(cols_to_delete,inplace=True,axis=1)

In [89]:
print('Starting data read from csv file.')
match_data_test = test

numeric_cols=['bathrooms','bedrooms','price','Distance','no_of_features','month']
print('Deleting columns containing huge missing values, Single valued items and id columns which are of no use.')
print('No data leakage is present as such.')
match_data_test.interest_level = match_data.interest_level.astype('category')

print('Using standard scaler of sklearn library to Standardize and scale the numeric inputs.')
#Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
match_data_test[numeric_cols] = scaler.fit_transform(match_data_test[numeric_cols])
 

print('\nFinding Optimal parameters for 4 different models using GridSearch.')
varToUse = match_data_test.columns.tolist()
Train_X = generateX(match_data_test,varToUse)

Starting data read from csv file.
Deleting columns containing huge missing values, Single valued items and id columns which are of no use.
No data leakage is present as such.
Using standard scaler of sklearn library to Standardize and scale the numeric inputs.

Finding Optimal parameters for 4 different models using GridSearch.


In [87]:
def generateX(df,var_list):
   
    #make a copy of the required subset and drop rows containing Na
    mdata = df[var_list].copy()
    mdata.dropna(inplace=True)
   
    #Now, process data and create dummy variables if required with final data in Xvars
    import pandas
    Xvars = pandas.DataFrame()
    import pandas as pd
    for cols in var_list:
        if (str(mdata[cols].dtype) == 'category'):
            dummySer= pd.get_dummies(mdata[cols],prefix=cols+'_')
            Xvars = pd.concat([Xvars,dummySer],axis=1)
        else:
            Xvars =  pd.concat([Xvars,mdata[cols]],axis=1)
   
    X = Xvars.values.tolist()
    return X

In [90]:
# Calculate predictions: predictions
predictions = model.predict(Train_X)

In [91]:
predictions.shape

(74659, 3)

In [93]:
import pandas as pd
predictions_test=pd.DataFrame(predictions)
predictions_test.to_csv("predictions_rent.csv")

Kaggle Code:

In [141]:
import os
import sys
import operator
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [151]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=1000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [197]:
data_path = "../input/"
train_file = "train.json"
test_file ="test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)

(49352, 15)
(74659, 14)


In [198]:
features_to_use  = ["bathrooms", "bedrooms", "latitude", "longitude", "price"]
#features_to_use  = ["bathrooms", "bedrooms", "price"]

In [200]:
# count of photos #
train_df["num_photos"] = train_df["photos"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

a=[]
for index, row in train_df.iterrows():
    a.append(distance(row['latitude'],row['longitude']))
train_df["Distance"]=a

a=[]
for index, row in test_df.iterrows():
    a.append(distance(row['latitude'],row['longitude']))
test_df["Distance"]=a


# count of "features" #
train_df["num_features"] = train_df["features"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)

# count of words present in description column #
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

# convert the created column to datetime object so as to extract more features 
train_df["created"] = pd.to_datetime(train_df["created"])
test_df["created"] = pd.to_datetime(test_df["created"])

# Let us extract some features like year, month, day, hour from date columns #
train_df["created_year"] = train_df["created"].dt.year
test_df["created_year"] = test_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
test_df["created_month"] = test_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
test_df["created_day"] = test_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour
test_df["created_hour"] = test_df["created"].dt.hour

# adding all these new features to use list #
#features_to_use.extend(["num_photos", "num_features", "num_description_words","created_year", "created_month", "created_day", "listing_id", "created_hour"])
features_to_use.extend(["num_photos", "num_features", "num_description_words", "Distance","listing_id","created_month", "created_day",  "created_hour"])

In [201]:
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train_df[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            features_to_use.append(f)

In [202]:
train_df['features'] = train_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
test_df['features'] = test_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
print(train_df["features"].head())
tfidf = CountVectorizer(stop_words='english', max_features=150)
tr_sparse = tfidf.fit_transform(train_df["features"])
te_sparse = tfidf.transform(test_df["features"])

10                                                         
10000     Doorman Elevator Fitness_Center Cats_Allowed D...
100004    Laundry_In_Building Dishwasher Hardwood_Floors...
100007                               Hardwood_Floors No_Fee
100013                                              Pre-War
Name: features, dtype: object


In [203]:
#train_df['description'] = train_df["description"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
#test_df['description'] = test_df["description"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
print(train_df["description"].head())
tfidf = CountVectorizer(stop_words='english', max_features=100)
tr_sparse1 = tfidf.fit_transform(train_df["description"])
te_sparse1 = tfidf.transform(test_df["description"])

10        A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...
10000                                                      
100004    Top Top West Village location, beautiful Pre-w...
100007    Building Amenities - Garage - Garden - fitness...
100013    Beautifully renovated 3 bedroom flex 4 bedroom...
Name: description, dtype: object


In [204]:
train_X = sparse.hstack([train_df[features_to_use], tr_sparse,tr_sparse1]).tocsr()
test_X = sparse.hstack([test_df[features_to_use], te_sparse,te_sparse1]).tocsr()

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))
print(train_X.shape, test_X.shape)

(49352, 267) (74659, 267)


In [205]:
cv_scores = []
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(train_X.shape[0])):
        dev_X, val_X = train_X[dev_index,:], train_X[val_index,:]
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        preds, model = runXGB(dev_X, dev_y, val_X, val_y)
        cv_scores.append(log_loss(val_y, preds))
        print(cv_scores)
        break

[0]	train-mlogloss:1.03707	test-mlogloss:1.03848
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:0.987857	test-mlogloss:0.990305
[2]	train-mlogloss:0.946093	test-mlogloss:0.949247
[3]	train-mlogloss:0.908338	test-mlogloss:0.91239
[4]	train-mlogloss:0.873321	test-mlogloss:0.878494
[5]	train-mlogloss:0.845121	test-mlogloss:0.851339
[6]	train-mlogloss:0.819193	test-mlogloss:0.82629
[7]	train-mlogloss:0.795103	test-mlogloss:0.803063
[8]	train-mlogloss:0.775901	test-mlogloss:0.784657
[9]	train-mlogloss:0.760272	test-mlogloss:0.769731
[10]	train-mlogloss:0.743722	test-mlogloss:0.754087
[11]	train-mlogloss:0.729279	test-mlogloss:0.740497
[12]	train-mlogloss:0.714859	test-mlogloss:0.72706
[13]	train-mlogloss:0.702144	test-mlogloss:0.715106
[14]	train-mlogloss:0.689939	test-mlogloss:0.703663
[15]	train-mlogloss:0.679837	test-mlogloss:0.694461
[16]	train-mlogloss:0.671101	tes

In [207]:
preds, model = runXGB(train_X, train_y, test_X, num_rounds=400)

In [208]:
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("xgb_starter2.csv", index=False)